In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy
!pip install kss
%cd /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
!pwd
from frameBERT import frame_parser
path="/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 8.1MB/s 
     |████████████████████████████████| 890kB 15.3MB/s 
     |████████████████████████████████| 3.0MB 15.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f61eaf45a057afa8b4f8282875c13f486d395a437159cd37b5b1625024db9f6a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635

In [50]:
import kss
from konlpy.tag import Hannanum
import re
import time
import numpy as np
import heapq
import pandas as pd
from operator import itemgetter 

In [3]:
text ='''
TEXT2PPTX의 구현하기 위해 two track process를 거쳤습니다. 대본을 피피티로 옮기기 전에 요약하고 분석하기 위해 자연어처리의 최신 기술을 다수 사용하였습니다. 또한 파이썬에서 파워포인트의 소스에 접근하기 위해 xml 코드를 심층적으로 분석했습니다. 그 결과 사용자가 자연어로 쓰인 대본을 텍2피에 제공하면 높은 수준의 피피티로 제공할 수 있게 되었습니다.
'''

In [36]:
ls = kss.split_sentences(text)
print("🧐 Numbers of sentences ",len(ls))
parser = frame_parser.FrameParser(model_path=path, language='ko')

🧐 Numbers of sentences  4
srl model: framenet
language: ko
version: 1.2
using viterbi: False
using masking: True
pretrained BERT: bert-base-multilingual-cased
using TGT special token: True
used dictionary:
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lu2idx.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lufrmap.json
	 /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/mul_bio_frargmap.json
...loaded model path: /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
...model is loaded


/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


In [25]:
def split(object):
  ls = kss.split_sentences(object)
  return ls

def parse(text):
  parser = frame_parser.FrameParser(model_path=path, language='ko')
  parsed = parser.parser(text, sent_id='1', result_format='conll')
  
def findBegin(parsed):
  a = [0]*10
  for _ in range(len(parsed)):
    a[_]=parsed
    tagged = a[_][3]
    words = a[_][0]
    for element in tagged:
      swb = element.startswith("B")
      if swb:
        b = tagged[swb]
        
        idx = tagged.index(b)-1
        role = tagged[idx].split("-")[1]
        beginning = words[idx]

  


### 첫 번째 문장은 이미 summarization에 의해 포함된 문장이므로 relation extraction에 의해서 이중으로 포함시키지 않는 것이 맞다.

In [131]:
h.pos(ls[0])

[('TEXT2PPTX', 'N'),
 ('의', 'J'),
 ('구현', 'N'),
 ('하', 'X'),
 ('기', 'E'),
 ('위하', 'P'),
 ('어', 'E'),
 ('two', 'F'),
 ('track', 'F'),
 ('process', 'F'),
 ('를', 'J'),
 ('거치', 'P'),
 ('었습니다', 'E'),
 ('.', 'S')]

In [114]:
# 1st sentence : GOALS & MEANS 

parsed = parser.parser(ls[0], sent_id='1', result_format='conll')
words = parsed[0][0]
print("😀 Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count


print("Number of parsed candidates ",len(parsed))
print(q, heapq.nlargest(2, q)) # 첫번째 경우 사용

words = parsed[0][0]
roles = parsed[0][2]
tagged = parsed[0][3]
for idx in range(len(tagged)):
  try:
    roles[i] = tagged[i].split("-")[1]
  except:
    roles[i] = tagged[i].split("-")[0]

print("roles ",roles)

goals = []
means = []
for _ in range(len(words)):
  if roles[_]=="Goal":
    goals.append(words[_])
  if roles[_]=='Means':
    means.append(words[_])

# MEANS
MEANS = ' '.join(means)
pos = h.pos(MEANS)
imp = []
for tup in h.pos(MEANS):
  if tup[1]=="N":
    imp.append(tup[0])
MEANS_TRUNCATED = ' '.join(imp)
print(MEANS_TRUNCATED)

# GOALS
GOAL = ' '.join(goals)
pos = h.pos(GOAL)
imp = []
for tup in h.pos(GOAL):
  if tup[1]=="N":
    imp.append(tup[0])
GOAL_TRUNCATED = ' '.join(imp)
print(GOAL_TRUNCATED)

final_string = MEANS_TRUNCATED + ' -- >' + GOAL_TRUNCATED
print("😀 final string to be inserted",final_string)

  



/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


😀 Words vector from sentence  ['TEXT2PPTX의', '구현하기', '위해', 'two', 'track', 'process를', '거쳤습니다.']
Number of parsed candidates  2
[5, 3] [5, 3]
roles  ['_', 'Goal', 'Purpose', '_', '_', '_', '_']

구현하기
😀 final string to be inserted  -- >구현하기


In [130]:
# 2nd sentence : PURPOSE & INSTRUMENT
parsed = parser.parser(ls[1], sent_id='1', result_format='conll')
words = parsed[0][0]
print("😀 Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count


print("Number of parsed candidates ",len(parsed))
print(q, heapq.nlargest(3, q))
print(itemgetter(*[1,5])(parsed))
candidates = itemgetter(*[1,8])(parsed)
for _ in candidates:
  words = _[0]
  tagged = _[3]
  print("tagged vector of candidates: ",tagged)
  roles = _[2]
  for i in range(len(parsed)):
    try:
      roles[i] = tagged[i].split("-")[1]
    except:
      roles[i] = tagged[i].split("-")[0]
  print(roles,f'Tagged words for the roles : {tagged}')
# roles are selected as the one with purpose and instrument

purpose = []
instrument = []
for _ in range(len(words)):
  if roles[_]=="Purpose":
    purpose.append(words[_])
    
  elif roles[_]=="Instrument":
    instrument.append(words[_])
    
  elif roles[_]!="_": # using 포함
    instrument.append(words[_])
print("Instrument: ",instrument)
print("Purpose: ",purpose)





# PURPOSE
PURPOSE = ' '.join(purpose)
pos = h.pos(PURPOSE)
imp = []
for tup in pos:
  if tup[1]=="N":
    imp.append(tup[0])
PURPOSE_TRUNCATED = ' '.join(imp)
print(PURPOSE_TRUNCATED)

# INSTRUMENT
INSTRUMENT = ' '.join(instrument)
pos = h.pos(INSTRUMENT)
imp = []
for tup in pos:
  if tup[1]=="N":
    imp.append(tup[0])
INST_TRUNCATED = ' '.join(imp)
print(INST_TRUNCATED)

final_string = INST_TRUNCATED+ ' ➡️' + PURPOSE_TRUNCATED 
print("😀 final string to be inserted ", final_string)

    
    

/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


😀 Words vector from sentence  ['대본을', '피피티로', '옮기기', '전에', '요약하고', '분석하기', '위해', '자연어처리의', '최신', '기술을', '다수', '사용하였습니다.']
Number of parsed candidates  9
[2, 11, 4, 0, 6, 1, 1, 1, 11] [11, 11, 6]
([['대본을', '피피티로', '옮기기', '전에', '요약하고', '분석하기', '위해', '자연어처리의', '최신', '기술을', '다수', '사용하였습니다.'], ['_', '_', '_', '전.n', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', 'Time_vector', '_', '_', '_', '_', '_', '_', '_', '_'], ['B-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'O', 'B-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event', 'I-Event']], [['대본을', '피피티로', '옮기기', '전에', '요약하고', '분석하기', '위해', '자연어처리의', '최신', '기술을', '다수', '사용하였습니다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '최신.n', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', 'Relative_time', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Focal_participant', 'O', 'O']])
tagged vector of candidates:  ['B-Landmark_event', 'I-Landmark_event', 'I-Landmark_event', 'O', 'B-Event

In [85]:
# 3rd sentence : Goal - Means
parsed = parser.parser(ls[2], sent_id='1', result_format='conll')
words = parsed[0][0]
print("Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count

print(q, heapq.nlargest(3, q))

a,b,c=parsed
tagged = b[3]
for i in range(len(tagged)):
  try:
    roles[i] = tagged[i].split("-")[1]
  except:
    roles[i] = tagged[i].split("-")[0]



goals = []
means = []
for _ in range(len(words)):
  if roles[_]=="Goal":
    goals.append(words[_])
  if roles[_]=='Means':
    means.append(words[_])

# MEANS
MEANS = ' '.join(means)
pos = h.pos(MEANS)
imp = []
for tup in h.pos(MEANS):
  if tup[1]=="N":
    imp.append(tup[0])
MEANS_TRUNCATED = ' '.join(imp)
print(MEANS_TRUNCATED)

# GOALS
GOAL = ' '.join(goals)
pos = h.pos(GOAL)
imp = []
for tup in h.pos(GOAL):
  if tup[1]=="N":
    imp.append(tup[0])
GOAL_TRUNCATED = ' '.join(imp)
print(GOAL_TRUNCATED)

final_string = MEANS_TRUNCATED + ' -- >' + GOAL_TRUNCATED
print("😀 final string to be inserted",final_string)


/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT/src/utils.py:279: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


Words vector from sentence  ['또한', '파이썬에서', '파워포인트의', '소스에', '접근하기', '위해', 'xml', '코드를', '심층적으로', '분석했습니다.']
[2, 8, 3] [8, 3, 2]
askdf
askdf
askdf
askdf
   
코드 심층적 분석
파이썬 파워포인트 소스 접근하기
😀 final string to be inserted 코드 심층적 분석-->파이썬 파워포인트 소스 접근하기


In [129]:
# # Fourth Sentence

# parsed = parser.parser(ls[3], sent_id='1', result_format='conll')
# words = parsed[0][0]
# print("😀 Words vector from sentence ",words)
# q = [-1]*len(parsed)
# for i in range(len(parsed)):
#   count=0
#   for element in parsed[i][3]:
#     if element.startswith("O"):
#       count+=1
#   q[i] = len(words) - count


# print("Number of parsed candidates ",len(parsed))
# print(q, heapq.nlargest(3, q))


candidates = itemgetter(*[0,7])(parsed)
for _ in candidates:
  words = _[0]
  tagged = _[3]
  print("tagged vector of candidates: ",tagged)
  roles = _[2]
  for i in range(len(tagged)):
    try:
      roles[i] = tagged[i].split("-")[1]
    except:
      roles[i] = tagged[i].split("-")[0]

  print(words,f'\nTagged words for the roles : ',roles)
  print()
# roles are selected as the one with effects only

tagged = parsed[0][3]
roles = parsed[0][2]
for i in range(len(tagged)):
    try:
      roles[i] = tagged[i].split("-")[1]
    except:
      roles[i] = tagged[i].split("-")[0]

print(words,'\nTagged words for the roles : ',roles)  
print()



purpose = []
instrument = []
effects = []
entity = []

for _ in range(len(words)):
  if roles[_]=="Purpose":
    purpose.append(words[_])
    
  if roles[_]=="Instrument":
    instrument.append(words[_])
  
  if roles[_]=="Effect":
    effects.append(words[_])
    
  
  if roles[_]=="Entity":
    entity.append(words[_])
    
  elif roles[_]!="_": # using 포함
    instrument.append(words[_])
print("Effects ",effects)


# Effects
EFFECT = ' '.join(effects)
pos = h.pos(EFFECT)
imp = []
for tup in pos:
  if tup[1]=="N":
    imp.append(tup[0])
EFFECT_TRUNCATED = ' '.join(imp)
print(EFFECT_TRUNCATED)

final_string = EFFECT_TRUNCATED
print("😀 final string to be inserted ", final_string)

    
    


tagged vector of candidates:  ['O', 'O', 'B-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect', 'I-Effect']
['그', '결과', '사용자가', '자연어로', '쓰인', '대본을', '텍2피에', '제공하면', '높은', '수준의', '피피티로', '제공할', '수', '있게', '되었습니다.'] 
Tagged words for the roles :  ['O', 'O', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect', 'Effect']

tagged vector of candidates:  ['O', 'O', 'B-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Event', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'O', 'O']
['그', '결과', '사용자가', '자연어로', '쓰인', '대본을', '텍2피에', '제공하면', '높은', '수준의', '피피티로', '제공할', '수', '있게', '되었습니다.'] 
Tagged words for the roles :  ['O', 'O', 'Entity', 'Entity', 'Entity', 'Entity', 'Entity', 'Entity', 'Event', 'Entity', 'Entity', 'Entity', 'Entity', 'O', 'O']

['그', '결과', '사용자가', '자연어로', '쓰인', '대본을', '텍2피에', '제공하면', '높은', '수준의', '피피티

In [ ]:
slide_1.placeholders.element[18][2][2][1][1].text = "폰트, 템플릿, 벡터 이미지 등을 보기쉬운 인터페이스로 추천" # 두 번째 설명
slide_1.placeholders.element[19][2][2][1][1].text = "양질의 발표를 위해 내용의 근거가 될 수 있는 chart와 table 그리고 image 추천 시스템" # 세 번째 설명
prs.save('test1.pptx')